<a href="https://colab.research.google.com/github/devmt04/CNNs-Models/blob/main/Classification/LeNet-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

In [4]:
train_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = True,
                                          transform = transforms.Compose([
                                                transforms.Resize((32,32)),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                          download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                        train = False,
                                        transform = transforms.Compose([
                                                transforms.Resize((32,32)),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                        download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 496kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.54MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.93MB/s]


In [4]:
class LeNet5(torch.nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
        torch.nn.BatchNorm2d(6),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
        torch.nn.BatchNorm2d(16),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = torch.nn.Linear(400, 120)
    self.relu = torch.nn.ReLU()
    self.fc1 = torch.nn.Linear(120, 84)
    self.relu1 = torch.nn.ReLU()
    self.fc2 = torch.nn.Linear(84, num_classes)


  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    out = self.relu(out)
    out = self.fc1(out)
    out = self.relu1(out)
    out = self.fc2(out)
    return out

In [5]:
torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model = LeNet5(num_classes).to(device)

In [7]:
model

LeNet5(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=400, out_features=120, bias=True)
  (relu): ReLU()
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [8]:
model.parameters

<bound method Module.parameters of LeNet5(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=400, out_features=120, bias=True)
  (relu): ReLU()
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)>

In [9]:
# for param in model.parameters():
#   print(param)

In [10]:
device

device(type='cuda')

In [16]:
cost = torch.nn.CrossEntropyLoss()

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [23]:
train_loader.dataset.data

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [24]:
total_step = len(train_loader)

In [25]:
total_step

938

In [31]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    #forward pass
    outputs = model(images)
    loss = cost(outputs, labels)

    #backward and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (i+1) % 400 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.1754
Epoch [1/10], Step [800/938], Loss: 0.0200
Epoch [2/10], Step [400/938], Loss: 0.0370
Epoch [2/10], Step [800/938], Loss: 0.0345
Epoch [3/10], Step [400/938], Loss: 0.0964
Epoch [3/10], Step [800/938], Loss: 0.0106
Epoch [4/10], Step [400/938], Loss: 0.0126
Epoch [4/10], Step [800/938], Loss: 0.0053
Epoch [5/10], Step [400/938], Loss: 0.0014
Epoch [5/10], Step [800/938], Loss: 0.0054
Epoch [6/10], Step [400/938], Loss: 0.0119
Epoch [6/10], Step [800/938], Loss: 0.1120
Epoch [7/10], Step [400/938], Loss: 0.0202
Epoch [7/10], Step [800/938], Loss: 0.0023
Epoch [8/10], Step [400/938], Loss: 0.0053
Epoch [8/10], Step [800/938], Loss: 0.0024
Epoch [9/10], Step [400/938], Loss: 0.0213
Epoch [9/10], Step [800/938], Loss: 0.0065
Epoch [10/10], Step [400/938], Loss: 0.0053
Epoch [10/10], Step [800/938], Loss: 0.0015


In [33]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 99.17 %


In [1]:
"""https://www.digitalocean.com/community/tutorials/writing-lenet5-from-scratch-in-python"""

'https://www.digitalocean.com/community/tutorials/writing-lenet5-from-scratch-in-python'

In [ ]:
"""
loss.backward() : calculate and stores the gradient of parameters in .grad attribute
                      calculate d(Loss)/d(parameters) through backpropogation

optimizer.step() : reads the .grad attributes and apply the update rules


  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):

    The outer loop: for epoch in range(num_epochs)
    → loops over full passes through the dataset.

    The inner l oop: for i, (images, labels) in enumerate(train_loader)
    → loops over batches of data.

train_loader is a PyTorch DataLoader.
  It automatically splits the full dataset into mini-batches.

  Each iteration (for i, (images, labels)) fetches one batch of:

      images: batch of input images

      labels: batch of corresponding labels


"""